# Getting Started with Julia in Colab/Jupyter
You can either run this notebook in Google Colab, or using Jupyter on your own machine.

## Running on Google Colab
1. Work on a copy of this notebook: _File_ > _Save a copy in Drive_ (you will need a Google account). Alternatively, you can download the notebook using _File_ > _Download .ipynb_, then upload it to [Colab](https://colab.research.google.com/).
2. Execute the following cell (click on it and press Ctrl+Enter) to install Julia, IJulia (the Jupyter kernel for Julia) and other packages. You can update `JULIA_VERSION` and the other parameters, if you know what you're doing. Installation takes 2-3 minutes.
3. Reload this page (press Ctrl+R, or ⌘+R, or the F5 key) and continue to the _Checking the Installation_ section.

* _Note_: If your Colab Runtime gets reset (e.g., due to inactivity), repeat steps 2 and 3.

In [ ]:
%%shell
set -e

#---------------------------------------------------#
JULIA_VERSION="1.6.0" # any version ≥ 0.7.0
JULIA_PACKAGES="IJulia CSV DataFrames Pipe"
JULIA_PACKAGES_IF_GPU="CUDA"
JULIA_NUM_THREADS=4
#---------------------------------------------------#

if [ -n "$COLAB_GPU" ] && [ -z `which julia` ]; then
  # Install Julia
  JULIA_VER=`cut -d '.' -f -2 <<< "$JULIA_VERSION"`
  echo "Installing Julia $JULIA_VERSION on the current Colab Runtime..."
  BASE_URL="https://julialang-s3.julialang.org/bin/linux/x64"
  URL="$BASE_URL/$JULIA_VER/julia-$JULIA_VERSION-linux-x86_64.tar.gz"
  wget -nv $URL -O /tmp/julia.tar.gz # -nv means "not verbose"
  tar -x -f /tmp/julia.tar.gz -C /usr/local --strip-components 1
  rm /tmp/julia.tar.gz

  # Install Packages
  if [ "$COLAB_GPU" = "1" ]; then
      JULIA_PACKAGES="$JULIA_PACKAGES $JULIA_PACKAGES_IF_GPU"
  fi
  for PKG in `echo $JULIA_PACKAGES`; do
    echo "Installing Julia package $PKG..."
    julia -e 'using Pkg; pkg"add '$PKG'; precompile;"' &> /dev/null
  done

  # Install kernel and rename it to "julia"
  echo "Installing IJulia kernel..."
  julia -e 'using IJulia; IJulia.installkernel("julia", env=Dict(
      "JULIA_NUM_THREADS"=>"'"$JULIA_NUM_THREADS"'"))'
  KERNEL_DIR=`julia -e "using IJulia; print(IJulia.kerneldir())"`
  KERNEL_NAME=`ls -d "$KERNEL_DIR"/julia*`
  mv -f $KERNEL_NAME "$KERNEL_DIR"/julia  

  echo ''
  echo "Successfully installed `julia -v`!"
  echo "Please reload this page (press Ctrl+R, ⌘+R, or the F5 key) then"
  echo "jump to the 'Checking the Installation' section."
fi

Installing Julia 1.6.0 on the current Colab Runtime...
2022-01-16 12:06:02 URL:https://storage.googleapis.com/julialang2/bin/linux/x64/1.6/julia-1.6.0-linux-x86_64.tar.gz [112838927/112838927] -> "/tmp/julia.tar.gz" [1]
Installing Julia package IJulia...
Installing Julia package CSV...
Installing Julia package DataFrames...


## Checking the Installation
The `versioninfo()` function should print your Julia version and some other info about the system (if you ever ask for help or file an issue about Julia, you should always provide this information).

In [ ]:
versioninfo()

Julia Version 1.6.0
Commit f9720dc2eb (2021-03-24 12:55 UTC)
Platform Info:
  OS: Linux (x86_64-pc-linux-gnu)
  CPU: Intel(R) Xeon(R) CPU @ 2.20GHz
  WORD_SIZE: 64
  LIBM: libopenlibm
  LLVM: libLLVM-11.0.1 (ORCJIT, broadwell)
Environment:
  JULIA_NUM_THREADS = 4


# Step 1. Import the necessary libraries

In [1]:
using DataFrames
using CSV
using Pipe
using Statistics

# Step 2. This is the data given as a dictionary

In [2]:
raw_data = Dict{Symbol,Array{Any}}(:regiment => ["Nighthawks", "Nighthawks", "Nighthawks", "Nighthawks", "Dragoons", "Dragoons", "Dragoons", "Dragoons", "Scouts", "Scouts", "Scouts", "Scouts"],
    :company => ["1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd"],
    :deaths => [523, 52, 25, 616, 43, 234, 523, 62, 62, 73, 37, 35],
    :battles => [5, 42, 2, 2, 4, 7, 8, 3, 4, 7, 8, 9],
    :size => [1045, 957, 1099, 1400, 1592, 1006, 987, 849, 973, 1005, 1099, 1523],
    :veterans => [1, 5, 62, 26, 73, 37, 949, 48, 48, 435, 63, 345],
    :readiness => [1, 2, 3, 3, 2, 1, 2, 3, 2, 1, 2, 3],
    :armored => [1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1],
    :deserters => [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3],
    :origin => ["Arizona", "California", "Texas", "Florida", "Maine", "Iowa", "Alaska", "Washington","Oregon", "Wyoming", "Louisana", "Georgia"])

Dict{Symbol, Array{Any, N} where N} with 10 entries:
  :company   => ["1st", "1st", "2nd", "2nd", "1st", "1st", "2nd", "2nd", "1st",…
  :readiness => [1, 2, 3, 3, 2, 1, 2, 3, 2, 1, 2, 3]
  :armored   => [1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 1]
  :veterans  => [1, 5, 62, 26, 73, 37, 949, 48, 48, 435, 63, 345]
  :deaths    => [523, 52, 25, 616, 43, 234, 523, 62, 62, 73, 37, 35]
  :origin    => ["Arizona", "California", "Texas", "Florida", "Maine", "Iowa", …
  :size      => [1045, 957, 1099, 1400, 1592, 1006, 987, 849, 973, 1005, 1099, …
  :deserters => [4, 24, 31, 2, 3, 4, 24, 31, 2, 3, 2, 3]
  :regiment  => ["Nighthawks", "Nighthawks", "Nighthawks", "Nighthawks", "Drago…
  :battles   => [5, 42, 2, 2, 4, 7, 8, 3, 4, 7, 8, 9]

# Step 3. Create a dataframe and assign it to a variable called army.

In [3]:
army = DataFrame(;raw_data...)

,company,readiness,armored,veterans,deaths,origin,size,deserters,regiment
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1st,1,1,1,523,Arizona,1045,4,Nighthawks
2,1st,2,0,5,52,California,957,24,Nighthawks
3,2nd,3,1,62,25,Texas,1099,31,Nighthawks
4,2nd,3,1,26,616,Florida,1400,2,Nighthawks
5,1st,2,0,73,43,Maine,1592,3,Dragoons
6,1st,1,1,37,234,Iowa,1006,4,Dragoons
7,2nd,2,0,949,523,Alaska,987,24,Dragoons
8,2nd,3,1,48,62,Washington,849,31,Dragoons
9,1st,2,0,48,62,Oregon,973,2,Scouts


# Step 5. Print only the column veterans

In [4]:
army[!,:veterans]

12-element Vector{Any}:
   1
   5
  62
  26
  73
  37
 949
  48
  48
 435
  63
 345

# Step 6. Print the columns 'veterans' and 'deaths'

In [5]:
army[!, [:veterans, :deaths]]

,veterans,deaths
,Any,Any
1,1,523
2,5,52
3,62,25
4,26,616
5,73,43
6,37,234
7,949,523
8,48,62
9,48,62


# Step 7. Print the name of all the columns.

In [6]:
names(army)

10-element Vector{String}:
 "company"
 "readiness"
 "armored"
 "veterans"
 "deaths"
 "origin"
 "size"
 "deserters"
 "regiment"
 "battles"

# Step 8. Select the 'deaths', 'size' and 'deserters' columns from Maine and Alaska


In [7]:
filter(x -> x[:origin] in ["Maine", "Alaska"], army)[!,[:origin,:deaths,:size,:deserters]]

,origin,deaths,size,deserters
,Any,Any,Any,Any
1,Maine,43,1592,3
2,Alaska,523,987,24


# Step 9. Select the rows 3 to 7 and the columns 3 to 6

In [8]:
army[3:7, 3:6]

,armored,veterans,deaths,origin
,Any,Any,Any,Any
1,1,62,25,Texas
2,1,26,616,Florida
3,0,73,43,Maine
4,1,37,234,Iowa
5,0,949,523,Alaska


# Step 10. Select every row after the fourth row and all columns

In [9]:
army[5:end, :]

,company,readiness,armored,veterans,deaths,origin,size,deserters,regiment
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1st,2,0,73,43,Maine,1592,3,Dragoons
2,1st,1,1,37,234,Iowa,1006,4,Dragoons
3,2nd,2,0,949,523,Alaska,987,24,Dragoons
4,2nd,3,1,48,62,Washington,849,31,Dragoons
5,1st,2,0,48,62,Oregon,973,2,Scouts
6,1st,1,0,435,73,Wyoming,1005,3,Scouts
7,2nd,2,1,63,37,Louisana,1099,2,Scouts
8,2nd,3,1,345,35,Georgia,1523,3,Scouts


# Step 11. Select every row up to the 4th row and all columns

In [10]:
army[1:4, :]

,company,readiness,armored,veterans,deaths,origin,size,deserters,regiment
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1st,1,1,1,523,Arizona,1045,4,Nighthawks
2,1st,2,0,5,52,California,957,24,Nighthawks
3,2nd,3,1,62,25,Texas,1099,31,Nighthawks
4,2nd,3,1,26,616,Florida,1400,2,Nighthawks


# Step 12. Select the 3rd column up to the 7th column

In [11]:
army[!, 3:7]

,armored,veterans,deaths,origin,size
,Any,Any,Any,Any,Any
1,1,1,523,Arizona,1045
2,0,5,52,California,957
3,1,62,25,Texas,1099
4,1,26,616,Florida,1400
5,0,73,43,Maine,1592
6,1,37,234,Iowa,1006
7,0,949,523,Alaska,987
8,1,48,62,Washington,849
9,0,48,62,Oregon,973


# Step 13. Select rows where df.deaths is greater than 50

In [12]:
filter(x -> x.deaths > 50, army)

,company,readiness,armored,veterans,deaths,origin,size,deserters,regiment
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1st,1,1,1,523,Arizona,1045,4,Nighthawks
2,1st,2,0,5,52,California,957,24,Nighthawks
3,2nd,3,1,26,616,Florida,1400,2,Nighthawks
4,1st,1,1,37,234,Iowa,1006,4,Dragoons
5,2nd,2,0,949,523,Alaska,987,24,Dragoons
6,2nd,3,1,48,62,Washington,849,31,Dragoons
7,1st,2,0,48,62,Oregon,973,2,Scouts
8,1st,1,0,435,73,Wyoming,1005,3,Scouts


# Step 14. Select rows where df.deaths is greater than 500 or less than 50

In [13]:
filter(x -> 50<x.deaths<500,army)

,company,readiness,armored,veterans,deaths,origin,size,deserters,regiment
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1st,2,0,5,52,California,957,24,Nighthawks
2,1st,1,1,37,234,Iowa,1006,4,Dragoons
3,2nd,3,1,48,62,Washington,849,31,Dragoons
4,1st,2,0,48,62,Oregon,973,2,Scouts
5,1st,1,0,435,73,Wyoming,1005,3,Scouts


# Step 15. Select all the regiments not named "Dragoons"

In [14]:
filter(x -> x.regiment != "Dragoons", army)

,company,readiness,armored,veterans,deaths,origin,size,deserters,regiment
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1st,1,1,1,523,Arizona,1045,4,Nighthawks
2,1st,2,0,5,52,California,957,24,Nighthawks
3,2nd,3,1,62,25,Texas,1099,31,Nighthawks
4,2nd,3,1,26,616,Florida,1400,2,Nighthawks
5,1st,2,0,48,62,Oregon,973,2,Scouts
6,1st,1,0,435,73,Wyoming,1005,3,Scouts
7,2nd,2,1,63,37,Louisana,1099,2,Scouts
8,2nd,3,1,345,35,Georgia,1523,3,Scouts


# Step 16. Select the rows called Texas and Arizona

In [15]:
filter(x -> x[:origin] in ["Texas", "Arizona"], army)

,company,readiness,armored,veterans,deaths,origin,size,deserters,regiment
,Any,Any,Any,Any,Any,Any,Any,Any,Any
1,1st,1,1,1,523,Arizona,1045,4,Nighthawks
2,2nd,3,1,62,25,Texas,1099,31,Nighthawks


# Step 17. Select the third cell in the row named Arizona

In [18]:
filter(x->x[:origin] == "Arizona",army)[!,3]

1-element Vector{Any}:
 1

# Step 18. Select the third cell down in the column named deaths

In [19]:
army[!,:deaths][3]

25